In [ ]:
# !python -m rpy2.situation

### Documentation
-----------------

- Currently the setup requires that the use must have R downloaded 
- They must also have packages gtools, splines2, pracma, RSpectra installed in the R environment (this can be included in the install_fdascc func)
- run the set_r_home function by specifying the location where R is installed to set the r_home environ variable for rpy2 to use
- have the FDASCC folder saved on their machine
- run the install_fdascc function with the location of the FDASCC folder to download the R code as a package

In [2]:
from FDASCC_py.setup import set_r_home, install_fdascc
set_r_home(path_to_R='C:\Program Files\R\R-4.2.0')
# install_fdascc(path_to_fdascc="C:/Users/alley/Desktop/folders/courses-and-certifications/phd-cs/research/1d_fda/FDASCC_py/FDASCC/FDASCC")

r_home environment variable set to C:\Program Files\R\R-4.2.0


In [3]:
import json
import pandas as pd
import numpy as np
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.functions import SignatureTranslatedFunction as STM

In [4]:
utils = importr('utils')

In [5]:
installed_packages = utils.installed_packages()

In [6]:
installed_packages

'FDASCC','gtools','pracma',...,'4.2.0','4.2.0','4.2.0'


In [7]:
fdascc = importr('FDASCC')

In [8]:
## example 1
ro.r('''
n <- 50
N <- 100
# N <- 2*n^(1/4)*log(n)
X <- seq(1/N,1,1/N)
X.band <- seq(1/(N-1), 1, 1/(N-1))
Y <- matrix(0,n,N)
mu.true <- 5*X + 4*sin(2*pi*(X-0.5))
psi1 <- -sqrt(2)*cos(2*pi*(X-0.5))
psi2 <- sqrt(2) * sin(4*pi*(X-0.5))
sig <- 0.1
lam1 <- 2
lam2 <- 1
xi1 <- rnorm(n, 0, sqrt(lam1))
xi2 <- rnorm(n, 0, sqrt(lam2))
for(i in 1:n){
  # Y[i,]=10*sin(X)+(rnorm(N, 0, sig))
  Y[i,] <- mu.true + xi1[i]*psi1 + xi2[i]*psi2 + (rnorm(N, 0, sig))
}
''')

X = ro.r['X']
X_band = ro.r['X.band']
Y = ro.r['Y']

out = fdascc.scc_1D(Ya=Y, X=X, X_band=X_band)

In [17]:
plot = ro.r['plot']

In [ ]:
plot(out[0])

In [9]:
#example 2
ro.r('''
n <- 100
N <- 2*n^(1/4)*log(n)
X <- seq(1/N,1,1/N)
X.band <- seq(0.01, 1, 0.01)
mu1.true <- 10 + sin(2*pi*(X-1/2))
psi1 <- -sqrt(2)*cos(pi*(X-1/2))
psi2 <- sqrt(2)*sin(pi*(X-1/2))
lam1 <- 2
lam2 <- 0.5
sig <- 0.3  # sig <- 0.5
xi1 <- rnorm(n, 0, sqrt(lam1))
xi2 <- rnorm(n, 0, sqrt(lam2))
Y <- matrix(0,n,N)
for(i in 1:n){
  # Y[i,]=10*sin(X)+(rnorm(N, 0, sig))
  Y[i,] <- mu1.true + xi1[i]*psi1 + xi2[i]*psi2 + (rnorm(N, 0, sig))
}
''')

X = ro.r['X']
X_band = ro.r['X.band']
Y = ro.r['Y']

out = fdascc.scc_1D(Ya=Y, X=X, X_band=X_band)

In [12]:
## Example 3: Two Sample Case for functional mean (Cao, Yang, Todem, 2012. Simulation Studies)
ro.r('''
na <- 80
nb <- 160
N <- 100
X <- seq(1/N,1,1/N)
X.band <- seq(0.01, 1, 0.01)
mu1.true <- 10 + sin(2*pi*(X-1/2))
diff.mu <- 0.6 * X
# diff.mu <- 0.7 * sin(X)
# diff.mu <- 0
mu2.true <- mu1.true + diff.mu
psi1 <- -sqrt(2)*cos(pi*(X-1/2))
psi2 <- sqrt(2)*sin(pi*(X-1/2))
lam1 <- 2
lam2 <- 0.5
sig <- 0.3  # sig <- 0.5
xi1.all <- list(rnorm(na,mean=0,sd=sqrt(lam1)),
            rnorm(nb,mean=0,sd=sqrt(lam1)))
xi2.all <- list(rnorm(na,mean=0,sd=sqrt(lam2)),
            rnorm(nb,mean=0,sd=sqrt(lam2)))
Ya <- matrix(0,na,N)
Yb <- matrix(0,nb,N)
for(i in 1:na){
  xi1 <- xi1.all[[1]]
  xi2 <- xi2.all[[1]]
  Ya[i,] <- mu1.true + xi1[i]*psi1 + xi2[i]*psi2 + (rnorm(N, 0, sig))
}
for(i in 1:nb){
  xi1 <- xi1.all[[2]]
  xi2 <- xi2.all[[2]]
  Yb[i,] <- mu2.true + xi1[i]*psi1 + xi2[i]*psi2 + (rnorm(N, 0, sig))
}
''')

X = ro.r['X']
X_band = ro.r['X.band']
Ya = ro.r['Ya']
Yb = ro.r['Yb']

out = fdascc.scc_1D(Ya, Yb, X=X, X_band=X_band)

In [15]:
out[0]

9.757597,9.909963,9.696268,...,10.718016,10.206988,10.658102
